# 实验七：基于序列模型的中英文翻译机



## 7.1 英中机器翻译实验导览



本 Notebook 以简单的中英句子翻译任务为例，完整呈现一个机器学习实验从数据准备到模型评估的基本流程，重点面向刚学习完机器学习导论、尚无工程实践经验的同学。



### 7.1.1 环境快速检查清单（建议先全部通过再往下走）



在开始运行代码前，先按照下面的检查单逐项确认环境是否就绪。若有一步失败，请先解决再继续，以免后面在训练阶段才发现环境问题。



**（1）操作系统与 Python 运行环境**



- 操作系统：支持 Windows / macOS / Linux，推荐按照课堂示例使用 Linux 或带 VS Code 的桌面环境；

- Python 版本：**必须使用 Python 3.8–3.10 之间的版本**（`pzmllab` 暂不支持 3.10 以上版本），可用 `python --version` / `python3 --version` 检查；

- 推荐统一使用 **Miniconda** 管理虚拟环境，便于课堂说明和助教协助排查；

  - 搜索关键词：`安装 Miniconda`、`conda 创建虚拟环境 python=3.10`；

- Jupyter 环境：可以正常启动并打开本 Notebook（推荐在 VS Code 中通过 Jupyter 扩展运行）。



**（2）快速安装本实验所需依赖（面向学生）**



- 在本仓库 `lab7/` 目录下已经提供精简版依赖文件：`requirements.txt`；

- 建议在新建好的虚拟环境中直接执行：

  - 使用 pip：`pip install -r requirements.txt`

- 该精简文件只包含本实验 Notebook 实际需要的核心包：

  - `torch`：用于搭建与训练序列模型；

  - `numpy`：基础数值计算；

  - `matplotlib`：绘制损失曲线与注意力热力图；

  - `nltk`：计算 BLEU 分数；

  - `pzmllog`：JetML 实验日志上报（仅在使用 JetML 平台时需要）。

- 若安装过程中遇到网络或镜像问题，可搜索：`pip 换国内源`、`conda 环境 配置 教程` 等关键词。



> 说明：仓库中可能还包含一个较长的完整依赖快照文件（例如 `requirements.full-freeze.txt`），那是用于老师/助教复现实验环境的参考，不是给学生直接使用的安装列表。



**（3）数据与文件路径**



- 确认当前工作目录下存在 `./data/eng-cmn.txt`，且编码为 UTF-8；

- 若使用 JetML，请确认本地已正确配置 `pzmllog` 所依赖的配置文件 / 服务（详见平台使用说明）；

- 若在远程服务器上运行，确保对 `data/` 目录有读权限，对日志 / 模型保存目录有写权限。



**（4）GPU / CPU 检查**



- 运行本 Notebook 中的“环境检查”代码单元，观察输出的 `device`：

  - 若显示 `cuda`，说明已成功识别 GPU；

  - 若显示 `cpu`，也可以完成实验，只是训练时间相对更长。

- 无论是否有 GPU，本实验都可以在较小迭代次数下完成主要体验。



**（5）快速自检：首个代码单元应当能够顺利运行**



- 依次运行下方“环境检查、依赖导入”单元：应当可以正常导入所有包，并输出 `device`；

- 若在导入某个包时卡住或报错（例如 `ImportError` / `ModuleNotFoundError`），请先根据错误信息安装或修复对应依赖，再继续后续步骤；

- 建议在环境稳定后再开始长时间训练，以避免中途因为环境问题中断。



**（6）编辑器与阅读体验建议（推荐但非必须）**



- 推荐统一使用 **VS Code** 打开本实验仓库，便于与课堂演示保持一致；

- 建议安装 VS Code 官方的 Python / Jupyter 扩展（可搜索 `VS Code 安装 Python 插件`）；

- 在 VS Code 的“**大纲视图（Outline）**”中可以直接看到本 Notebook 的各级标题结构，便于按照“数据准备→模型→训练→评估”的顺序导航；

- 若使用纯浏览器 Jupyter，也可以借助目录扩展（如 Table of Contents）获得类似的大纲视图。



### 7.1.2 学习目标



完成本实验后，期望你能够：

- **数据层面**：理解如何将原始中英文句子经过清洗和编码，转换为模型可以处理的数值表示；

- **模型层面**：理解一个“先压缩句子信息、再逐词生成译文”的神经网络结构的整体思路；

- **训练层面**：理解通过多轮迭代、损失下降来逐步提升模型性能的基本过程；

- **评估层面**：能够结合数值指标和可视化结果，对模型输出质量进行定性与定量分析。



### 7.1.3 章节结构概览



1. **环境与工具**：导入依赖库，配置日志记录与计时辅助函数；

2. **Step1 数据准备**：构建词表，对句子进行标准化处理，读取并筛选语料，统计词频，并通过样例检查数据是否正确；

3. **Step2 模型设计**：定义将句子表示为向量的子网络、根据向量逐步生成译文的子网络，以及若干句子与张量之间转换的辅助函数；

4. **Step3 模型训练**：给出单步训练函数和多轮训练循环，输出损失值并记录训练过程中的关键信息；

5. **Step4 模型评估**：对单句和随机样本进行测试，计算评价分数，并绘制训练曲线与“模型关注位置”的可视化图像；

6. **Main Execution**：集中实例化前面定义的组件，**在这一节真正串联并执行“数据准备→模型→训练→评估”这四个流程**；

7. **可视化扩展**：调整字体设置，展示多条句子的可视化结果，帮助更直观地理解模型行为。



> 提醒：前面的 Step1–Step4 是“搭建和拆解零部件（数据管线、模型结构、训练与评估函数）”，只有在最后的 **Main Execution** 单元里，这些函数才会被实际组合起来并跑完一次完整流程。



### 7.1.4 推荐阅读与执行顺序



- 首先运行“环境与工具”部分，确保依赖库与运行环境配置正确；

- 按照 Step1 中的小节顺序依次执行，直至能够打印出一对中英文句子，确认数据管线工作正常；

- 阅读 Step2，理解整体网络结构和数据在其中的流动方式；

- 在 Step3 中先进行较少轮次的试验性训练（例如 1,000 次迭代），观察损失值是否呈下降趋势；

- 若试验训练结果合理，再进行较大轮次的正式训练（例如 75,000 次迭代）；

- 之后进入 Step4 与可视化扩展部分，综合利用分数与图像对模型进行分析与反思。



### 7.1.5 运行时间预估（CPU 环境）



| 阶段           | 典型耗时   | 说明                         |
|----------------|------------|------------------------------|
| 数据准备       | < 1 min    | 语料规模较小，统计较为迅速   |
| 小规模训练     | 1–2 min    | 用于检查训练过程是否正常     |
| 全量训练       | 10–25+ min | 取决于硬件性能和超参数设置   |
| 评估与可视化   | 1–3 min    | 生成评价指标与可视化图像     |

在具备 GPU 的环境下，训练时间通常会显著缩短。



### 7.1.6 初学者操作提示



- 逐节运行，避免跳过导致变量未定义；

- 调整 `teacher_forcing_ratio` 比较收敛差异 (0 vs 1)；

- 多次运行随机评估，观察 BLEU 波动范围；

- 注意力若只集中在序列边缘，多训练或增大数据；

- 若时间不足：降低迭代次数或隐藏维度。



### 7.1.7 可继续探索



- 换子词分词 (BPE) 降低未登录词；

- 尝试 Transformer 编码器替代 GRU；

- 加入学习率调度 / 早停策略；

- 比较不同注意力（点积 vs 可加性）。



准备好后，进入下一单元开始环境检查。祝实验顺利！


## 7.2 环境检查、依赖导入、辅助函数声明与实现

在动手之前，先把所有会用到的积木准备好：文本处理依赖 `unicodedata`、`re` 负责清洗句子，`torch` 负责搭建与训练 Seq2Seq 模型，`matplotlib` 用于绘图，`nltk` 的 BLEU 分数作为评估指标。运行本单元即可完成环境检查，并确认是否有 GPU 可用。

In [33]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
import jupyter
import matplotlib
import matplotlib.pyplot as plt
import nltk
from nltk.translate.bleu_score import sentence_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### JetML 平台配置
为了便于上传日志、记录超参数，本项目接入了 JetML。下面的代码展示如何设置访问令牌、实验名称与端口，并将学习率、批大小等信息同步出去，方便教学或验收。


In [ ]:
from pzmllog import NewLogger

learning_rate = 0.01
epoch = 1
batch_size = 1


log = NewLogger(
    config={
        #用户 ID
        'access_token':"r6qo97vwncn4ogiwnsq549bd",
        #项目 ID
        'project':"1863",
        #实验描述和说明信息
        "description":"test",
        #自定义实验名称
        "experiment_name":"TEST",
        #仓库 ID
        "repository_id":"d6ca066d95d3400fba88a7e9335892ec",
        # tomcat的启动端口
        'port': "5560"
    },
    #超参数集
    info = {
        "learning_rate": learning_rate,
        "epoch": epoch,
        "batch_size": batch_size
    }
)

ConnectionError: Client Connection Error,Check Client's Status Please


### 计时辅助函数
长时间训练时需要掌握进度感。下面的工具函数把秒数转换成“分钟 + 秒”，并根据当前迭代估算剩余时间，方便在实验记录中写下训练耗时。


In [ ]:
#This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## 7.3 Step1:  数据准备


In [ ]:

SOS_token = 0  # Start of Sentence 句子开始标记
EOS_token = 1  # End of Sentence 句子结束标记


class Lang:
    """
    语言词典类
    功能：为一种语言（中文或英文）建立词汇表
    - word2index: 单词 → 索引
    - index2word: 索引 → 单词
    - word2count: 单词出现次数统计
    """
    def __init__(self, name):
        self.name = name
        self.word2index = {}  # 单词到索引的映射
        self.word2count = {}  # 单词计数
        self.index2word = {0: "SOS", 1: "EOS"}  # 索引到单词的映射
        self.n_words = 2  # 初始有 2 个词（SOS 和 EOS）

    def addSentence(self, sentence):
        """添加一个句子中的所有单词到词典"""
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        """添加单个单词到词典"""
        if word not in self.word2index:
            # 新词：分配新索引
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            # 已有词：增加计数
            self.word2count[word] += 1

### 文本标准化：把句子清洗到统一格式
为了让模型专注在语义而不是标点或大小写差异，我们需要把原始句子逐步变成“全小写、带空格分隔、只保留核心字符”的形式。下面的函数先执行 Unicode 归一化，接着通过正则表达式删掉中英文的句号、感叹号等符号。这一步属于 **数据准备** 环节的第一道工序。

In [ ]:
def unicodeToAscii(s):
    return ''.join(
    c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn'
    )


# 其中normalizeString函数中的正则表达式需对应更改，否则会将中文单词替换成空格
def normalizeString(s):
    #变成小写，去掉前后空格
    s = s.lower().strip()
    # 通过有无空格分隔作为分隔符，判断是否为中文（不robust）
    if ' ' not in s:
        s = list(s)
        s = ' '.join(s)
    s = unicodeToAscii(s)  #将unicode变成ascii
    s = re.sub(r"([.。!！?？])", "", s)
    return s


### 读取平行语料并构建语言对象
`readLangs` 负责打开 `data/eng-cmn.txt`，逐行拆分成英文和中文句子对，再用刚才的标准化函数做清洗。根据 `reverse` 标志可以交换输入/输出语言，从而复用同一份代码处理“英→中”或“中→英”任务。执行下面的代码，即可得到两个 `Lang` 词典对象与原始句子对列表。

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    file_path = "./data/eng-cmn.txt"
    with open(file_path, encoding='utf-8') as file:
        lines = file.readlines()
    

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]


    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

### 句子长度筛选：控制学习难度
为了让初学者可以在有限算力下快速跑通实验，我们只保留长度不超过 `MAX_LENGTH` 的句子，并聚焦在常见的主谓结构（`eng_prefixes`）。这属于典型的数据清洗策略：先把学习目标收窄，再逐步拓展。

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

### 汇总数据准备流程
`prepareData` 将前面所有步骤串联：读文件→筛句子→统计词频→生成词典。运行后我们就拿到了后续所有环节共享的数据结构，是“数据准备”阶段的终点。

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs



### 测试词典是否正确构建
运行下面的单元会实际读取语料、构建词典并随机打印一个句子对。确认能够输出内容后，说明数据准备阶段已经完成，可以继续往模型部分推进。

In [ ]:
input_lang, output_lang, pairs = prepareData('eng', 'cmn', True)
print(random.choice(pairs))


Reading lines...
Read 24026 sentence pairs
Trimmed to 459 sentence pairs
Counting words...
Counted words:
cmn 684
eng 560
['我 正 在 找 一 個 助 手 ', 'i am looking for an assistant']


In [ ]:
'''file_path = "./data/eng-cmn.txt"
with open(file_path, encoding='utf-8') as file:
    lines = file.readlines()
pairs = [[normalizeString(l).split('\t')[:2]] for l in lines]
cn = []
eng = []
for p in pairs:
    p=np.array(p)
    eng.append([p[0,0]])
    cn.append([p[0,1]])'''

'file_path = "./data/eng-cmn.txt"\nwith open(file_path, encoding=\'utf-8\') as file:\n    lines = file.readlines()\npairs = [[normalizeString(l).split(\'\t\')[:2]] for l in lines]\ncn = []\neng = []\nfor p in pairs:\n    p=np.array(p)\n    eng.append([p[0,0]])\n    cn.append([p[0,1]])'

## 7.4 Step2:  模型设计


### 7.4.1 RNN Encoder 类：把输入句子压缩成语义向量

编码器负责读取“源语言”单词序列并逐步更新隐藏状态，最终把整句的信息压成一个上下文向量。这里使用 `Embedding` + 单层 `GRU`，是理解 Seq2Seq 的最小原型。

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### 7.4.2 RNNDecoder 类：注意力辅助的翻译器

解码器一边“看”编码器的输出序列（注意力），一边一步步生成“目标语言”单词。理解解码器的前向过程，有助于你弄清楚“翻译时模型到底在用哪些源单词的信息”。

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### 7.4.3 句子转张量：连接数据与模型

本节把前面整理好的句子和词表真正“喂”进 PyTorch：把单词索引组装成张量，并考虑 `SOS/EOS` 等特殊标记。理解这一步有助于你在之后修改模型或替换数据集时少踩坑。

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## 7.5 Step3:  模型训练

### 7.5.1 train() 模型训练函数

先从“单次参数更新”出发，拆开看清每一步：前向传播、计算损失、反向传播与参数更新。理解这一小步，有利于你之后阅读 `trainIters` 时不迷路。

### 7.5.2 trainIters：把单步训练串成完整流程

在前一小节打好“单步训练”基础后，这里通过循环若干 epoch，把训练过程串联起来；你可以在这里观察损失下降趋势，并理解各种训练超参数（学习率、epoch 数等）对结果的影响。

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

## 7.6 Step4:  模型评估
评估阶段分两步：
- `evaluate` 输入单个句子，返回预测词序列与注意力矩阵
- `evaluateRandomly` 则随机抽样多条句子并计算 BLEU，帮助我们量化模型性能。记得在训练完成后再运行这些单元

### 7.6.1 模型评估

这里主要通过 BLEU 等指标定量评估翻译质量，并给出若干示例翻译结果，帮助你从“数字”和“具体例句”两个角度理解模型的优缺点。

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        # log.Log({"epoch":epoch,"loss":loss,"accuracy":accuracy})
        log.Log({"epoch":epoch,"loss":loss})

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


### 7.6.2 绘制训练损失曲线

最后，我们通过 Matplotlib 把训练过程中的损失变化画成曲线，一眼就能看到模型是否收敛、有没有明显的过拟合/欠拟合趋势，也方便你在之后尝试不同超参数时进行对比。

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

### 7.6.1 绘制训练损失曲线
训练完毕后，用 `showPlot` 把损失随迭代的变化画出来，可以快速判断学习率是否合适、是否过拟合。这是“训练”与“评估”之间最直接的桥梁。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [ ]:
def evaluateRandomly(encoder, decoder, n=100):
    sum_scores = 0
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        w = []
        words = pair[1].strip(' ').split(' ')
        words.append('<EOS>')
        w.append(words)
        bleu_score = sentence_bleu(w, output_words)
        sum_scores += bleu_score
    print('The bleu_score is ', sum_scores/n)

## 7.7 Main Execution：执行训练

这一单元汇总了“模型 + 数据 + 训练函数”，实例化编码器/解码器后调用 `trainIters`。建议先从较小的 `n_iters` 试跑，确认损失下降后再把迭代次数提升至 75k，以节约时间。

In [ ]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

log.Run()
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
trainIters(encoder1, attn_decoder1, 1000, print_every=100, learning_rate=learning_rate)
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000, learning_rate=learning_rate)
log.End()

# 结束整个过程,提交实验结果
log.Submit()

NameError: name 'log' is not defined

### 7.7.1 随机抽取样本进行定性评估

训练完后，用 `evaluateRandomly` 抽取若干句子对，手动观察生成结果是否通顺，并结合 BLEU 分数判断模型稳定性。这一步补充了定量指标中可能遗漏的语义问题。

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

### 7.7.2 绘图字体设置

在中文环境下绘图时，Matplotlib 默认字体可能无法显示汉字。以下配置切换到楷体并修复负号显示问题，确保注意力热力图能够正确呈现。

In [ ]:
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

### 7.7.3 单句注意力热力图示例

执行下列代码即可指定一句中文输入、查看模型输出的英文单词，并用 `matshow` 可视化注意力矩阵，帮助理解模型在生成每个词时关注了哪些输入位置。

In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "你 只 是 玩")
print(output_words)
plt.matshow(attentions.numpy())
plt.show()

### 7.7.4 批量可视化多个句子的注意力

最后的工具函数封装了“输入句子 → 打印翻译 → 展示注意力热力图”的流程。挑选不同语义结构的句子运行，能直观看出模型在长句、问句等场景下的表现差异。

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("他 和 他 的 邻 居 相 处 ")

evaluateAndShowAttention("我 肯 定 他 会 成 功 的 ")

evaluateAndShowAttention("他 總 是 忘 記 事 情")

evaluateAndShowAttention("我 们 非 常 需 要 食 物 ")